# Adversarial Autoencoder

ref. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
# from keras.layers import merge
from keras.layers import Lambda, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K

Using TensorFlow backend.


# Encoder

In [3]:
def build_encoder(img_shape, latent_dim):
    # Encoder

    img = Input(shape=img_shape)

    h = Flatten()(img)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    mu = Dense(latent_dim)(h)
    log_var = Dense(latent_dim)(h)
    
    # NOTE: original code uses keras.layer.merge() function which is deprecated
    #     latent_repr = merge([mu, log_var],
    #                          mode=lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2),
    #                          output_shape=lambda p: p[0])
    
    # NOW: we use Lambda layer to do the same thing
    latent_repr = Lambda(lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2), 
                         output_shape=lambda p: p[0])([mu, log_var])   
    
    return Model(img, latent_repr)

# Generator

In [4]:
def build_decoder(img_shape, latent_dim):

    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    z = Input(shape=(latent_dim,))
    img = model(z)

    return Model(z, img)

# Discriminator

In [5]:
def build_discriminator(latent_dim):

    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation="sigmoid"))
    model.summary()

    encoded_repr = Input(shape=(latent_dim, ))
    validity = model(encoded_repr)

    return Model(encoded_repr, validity)

# Save images

In [6]:
def sample_images(epoch, decoder, latent_dim):
    r, c = 5, 5

    z = np.random.normal(size=(r*c, latent_dim))
    gen_imgs = decoder.predict(z)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

# Save model

In [7]:
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

def save_model(G, D):
    save(G, "aae_generator")
    save(D, "aae_discriminator")

# Train the GAN

In [8]:
def train(encoder, decoder, adversarial_autoencoder, D, latent_dim, epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        latent_fake = encoder.predict(imgs)
        latent_real = np.random.normal(size=(batch_size, latent_dim))

        # Train the discriminator
        d_loss_real = D.train_on_batch(latent_real, valid)
        d_loss_fake = D.train_on_batch(latent_fake, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Adversarial autoencoder
        # ---------------------

        # Train the combined stack
        g_loss = adversarial_autoencoder.train_on_batch(imgs, [imgs, valid])

        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images(epoch, decoder, latent_dim)

# main()

In [9]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 10

In [10]:
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Build and compile the discriminator
D = build_discriminator(latent_dim)
D.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 137,217
Trainable params: 137,217
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Build the encoder / decoder
encoder = build_encoder(img_shape, latent_dim)
decoder = build_decoder(img_shape, latent_dim)

img = Input(shape=img_shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 784)               402192    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
Total params: 670,480
Trainable params: 670,480
Non-trainable params: 0
________________________________________________

In [13]:
# The generator takes the image, encodes it and reconstructs it
# from the encoding
encoded_repr = encoder(img)
reconstructed_img = decoder(encoded_repr)

In [14]:
# For the adversarial_autoencoder model we will only train the generator
D.trainable = False

In [15]:
# The discriminator determines validity of the encoding
validity = D(encoded_repr)

In [16]:
# The adversarial_autoencoder model  (stacked generator and discriminator)
adversarial_autoencoder = Model(img, [reconstructed_img, validity])
adversarial_autoencoder.compile(loss=['mse', 'binary_crossentropy'],
    loss_weights=[0.999, 0.001],
    optimizer=optimizer)

In [19]:
epochs=5000
# epochs=20000
train(encoder, decoder, adversarial_autoencoder, D, 
      latent_dim, epochs=epochs, batch_size=32, sample_interval=200)

0 [D loss: 0.275131, acc: 84.38%] [G loss: 0.113960, mse: 0.110850]
1 [D loss: 0.402569, acc: 84.38%] [G loss: 0.105627, mse: 0.101540]
2 [D loss: 0.184208, acc: 92.19%] [G loss: 0.097344, mse: 0.092902]
3 [D loss: 0.224868, acc: 93.75%] [G loss: 0.098065, mse: 0.094662]
4 [D loss: 0.209831, acc: 89.06%] [G loss: 0.099300, mse: 0.095027]
5 [D loss: 0.318525, acc: 89.06%] [G loss: 0.100917, mse: 0.097351]
6 [D loss: 0.240272, acc: 89.06%] [G loss: 0.092810, mse: 0.089493]
7 [D loss: 0.410071, acc: 85.94%] [G loss: 0.097257, mse: 0.093769]
8 [D loss: 0.387782, acc: 85.94%] [G loss: 0.112199, mse: 0.108287]
9 [D loss: 0.333077, acc: 85.94%] [G loss: 0.103574, mse: 0.099652]
10 [D loss: 0.163502, acc: 92.19%] [G loss: 0.092204, mse: 0.088174]
11 [D loss: 0.361003, acc: 82.81%] [G loss: 0.106953, mse: 0.103202]
12 [D loss: 0.279574, acc: 87.50%] [G loss: 0.113301, mse: 0.109552]
13 [D loss: 0.239538, acc: 90.62%] [G loss: 0.097244, mse: 0.092381]
14 [D loss: 0.135207, acc: 95.31%] [G loss: 

119 [D loss: 0.373036, acc: 87.50%] [G loss: 0.093449, mse: 0.089518]
120 [D loss: 0.179783, acc: 93.75%] [G loss: 0.099579, mse: 0.095510]
121 [D loss: 0.271474, acc: 89.06%] [G loss: 0.090708, mse: 0.087278]
122 [D loss: 0.175006, acc: 92.19%] [G loss: 0.102995, mse: 0.098959]
123 [D loss: 0.282402, acc: 90.62%] [G loss: 0.104404, mse: 0.100622]
124 [D loss: 0.147636, acc: 92.19%] [G loss: 0.095532, mse: 0.091337]
125 [D loss: 0.277953, acc: 85.94%] [G loss: 0.097769, mse: 0.095021]
126 [D loss: 0.199013, acc: 90.62%] [G loss: 0.107477, mse: 0.103479]
127 [D loss: 0.178354, acc: 93.75%] [G loss: 0.086236, mse: 0.081953]
128 [D loss: 0.222577, acc: 90.62%] [G loss: 0.093848, mse: 0.090862]
129 [D loss: 0.264881, acc: 89.06%] [G loss: 0.089584, mse: 0.085531]
130 [D loss: 0.204769, acc: 93.75%] [G loss: 0.099600, mse: 0.096041]
131 [D loss: 0.348018, acc: 82.81%] [G loss: 0.095298, mse: 0.091576]
132 [D loss: 0.163645, acc: 92.19%] [G loss: 0.098639, mse: 0.094796]
133 [D loss: 0.35071

238 [D loss: 0.413259, acc: 84.38%] [G loss: 0.089636, mse: 0.086237]
239 [D loss: 0.389101, acc: 81.25%] [G loss: 0.091511, mse: 0.089063]
240 [D loss: 0.427250, acc: 87.50%] [G loss: 0.089453, mse: 0.085928]
241 [D loss: 0.385738, acc: 85.94%] [G loss: 0.107064, mse: 0.103966]
242 [D loss: 0.412581, acc: 84.38%] [G loss: 0.087482, mse: 0.084037]
243 [D loss: 0.447030, acc: 78.12%] [G loss: 0.095133, mse: 0.092490]
244 [D loss: 0.305538, acc: 89.06%] [G loss: 0.101401, mse: 0.098713]
245 [D loss: 0.316586, acc: 85.94%] [G loss: 0.093112, mse: 0.090290]
246 [D loss: 0.426655, acc: 84.38%] [G loss: 0.079644, mse: 0.077223]
247 [D loss: 0.433283, acc: 79.69%] [G loss: 0.109065, mse: 0.106562]
248 [D loss: 0.429020, acc: 79.69%] [G loss: 0.070737, mse: 0.068076]
249 [D loss: 0.405870, acc: 84.38%] [G loss: 0.095519, mse: 0.093013]
250 [D loss: 0.410867, acc: 85.94%] [G loss: 0.092015, mse: 0.089638]
251 [D loss: 0.157530, acc: 93.75%] [G loss: 0.091198, mse: 0.087649]
252 [D loss: 0.30010

362 [D loss: 0.469841, acc: 82.81%] [G loss: 0.096144, mse: 0.092806]
363 [D loss: 0.282562, acc: 89.06%] [G loss: 0.091026, mse: 0.088172]
364 [D loss: 0.440608, acc: 81.25%] [G loss: 0.097681, mse: 0.095275]
365 [D loss: 0.505032, acc: 78.12%] [G loss: 0.090772, mse: 0.087958]
366 [D loss: 0.300411, acc: 87.50%] [G loss: 0.088660, mse: 0.085471]
367 [D loss: 0.363703, acc: 84.38%] [G loss: 0.088663, mse: 0.085971]
368 [D loss: 0.293487, acc: 89.06%] [G loss: 0.085397, mse: 0.082122]
369 [D loss: 0.410187, acc: 79.69%] [G loss: 0.101484, mse: 0.098722]
370 [D loss: 0.328396, acc: 87.50%] [G loss: 0.096250, mse: 0.094011]
371 [D loss: 0.281686, acc: 84.38%] [G loss: 0.096935, mse: 0.094146]
372 [D loss: 0.346703, acc: 85.94%] [G loss: 0.100604, mse: 0.097920]
373 [D loss: 0.443595, acc: 81.25%] [G loss: 0.102040, mse: 0.099624]
374 [D loss: 0.252083, acc: 87.50%] [G loss: 0.088284, mse: 0.084897]
375 [D loss: 0.263585, acc: 87.50%] [G loss: 0.080663, mse: 0.077192]
376 [D loss: 0.53904

480 [D loss: 0.393147, acc: 87.50%] [G loss: 0.084578, mse: 0.082523]
481 [D loss: 0.450243, acc: 81.25%] [G loss: 0.087862, mse: 0.085395]
482 [D loss: 0.337034, acc: 82.81%] [G loss: 0.094258, mse: 0.091631]
483 [D loss: 0.286249, acc: 85.94%] [G loss: 0.094338, mse: 0.091534]
484 [D loss: 0.412185, acc: 78.12%] [G loss: 0.092670, mse: 0.089906]
485 [D loss: 0.264616, acc: 92.19%] [G loss: 0.092686, mse: 0.090451]
486 [D loss: 0.384661, acc: 81.25%] [G loss: 0.086181, mse: 0.084002]
487 [D loss: 0.299030, acc: 92.19%] [G loss: 0.081962, mse: 0.079299]
488 [D loss: 0.369907, acc: 87.50%] [G loss: 0.091592, mse: 0.089198]
489 [D loss: 0.346031, acc: 84.38%] [G loss: 0.084682, mse: 0.081837]
490 [D loss: 0.283160, acc: 84.38%] [G loss: 0.090078, mse: 0.087310]
491 [D loss: 0.423301, acc: 81.25%] [G loss: 0.095312, mse: 0.093059]
492 [D loss: 0.201761, acc: 92.19%] [G loss: 0.094489, mse: 0.092240]
493 [D loss: 0.385174, acc: 85.94%] [G loss: 0.097070, mse: 0.094581]
494 [D loss: 0.42707

...

4515 [D loss: 0.406034, acc: 84.38%] [G loss: 0.069596, mse: 0.067887]
4516 [D loss: 0.469356, acc: 76.56%] [G loss: 0.061817, mse: 0.060221]
4517 [D loss: 0.497484, acc: 78.12%] [G loss: 0.085049, mse: 0.083735]
4518 [D loss: 0.387117, acc: 82.81%] [G loss: 0.068678, mse: 0.066808]
4519 [D loss: 0.480800, acc: 75.00%] [G loss: 0.071474, mse: 0.069756]
4520 [D loss: 0.518365, acc: 70.31%] [G loss: 0.075949, mse: 0.074800]
4521 [D loss: 0.584835, acc: 75.00%] [G loss: 0.070325, mse: 0.069054]
4522 [D loss: 0.421351, acc: 84.38%] [G loss: 0.071815, mse: 0.070300]
4523 [D loss: 0.332957, acc: 79.69%] [G loss: 0.061581, mse: 0.059601]
4524 [D loss: 0.444803, acc: 84.38%] [G loss: 0.088159, mse: 0.086529]
4525 [D loss: 0.462495, acc: 81.25%] [G loss: 0.073703, mse: 0.072191]
4526 [D loss: 0.501767, acc: 75.00%] [G loss: 0.068840, mse: 0.067179]
4527 [D loss: 0.539720, acc: 70.31%] [G loss: 0.083689, mse: 0.082457]
4528 [D loss: 0.444278, acc: 78.12%] [G loss: 0.085714, mse: 0.084355]
4529 [

4635 [D loss: 0.487924, acc: 79.69%] [G loss: 0.066501, mse: 0.064757]
4636 [D loss: 0.351703, acc: 89.06%] [G loss: 0.060602, mse: 0.059011]
4637 [D loss: 0.386755, acc: 85.94%] [G loss: 0.075040, mse: 0.073389]
4638 [D loss: 0.401162, acc: 84.38%] [G loss: 0.070153, mse: 0.068602]
4639 [D loss: 0.438250, acc: 76.56%] [G loss: 0.086735, mse: 0.085202]
4640 [D loss: 0.472525, acc: 76.56%] [G loss: 0.078661, mse: 0.077376]
4641 [D loss: 0.441324, acc: 81.25%] [G loss: 0.081645, mse: 0.080135]
4642 [D loss: 0.377244, acc: 87.50%] [G loss: 0.066213, mse: 0.064604]
4643 [D loss: 0.546070, acc: 75.00%] [G loss: 0.064677, mse: 0.063072]
4644 [D loss: 0.405006, acc: 81.25%] [G loss: 0.066571, mse: 0.064933]
4645 [D loss: 0.500796, acc: 76.56%] [G loss: 0.072483, mse: 0.071115]
4646 [D loss: 0.457046, acc: 76.56%] [G loss: 0.071716, mse: 0.070111]
4647 [D loss: 0.318292, acc: 92.19%] [G loss: 0.061251, mse: 0.059329]
4648 [D loss: 0.470568, acc: 84.38%] [G loss: 0.073972, mse: 0.072204]
4649 [

4756 [D loss: 0.477503, acc: 81.25%] [G loss: 0.071418, mse: 0.070014]
4757 [D loss: 0.480373, acc: 81.25%] [G loss: 0.066164, mse: 0.064812]
4758 [D loss: 0.401663, acc: 78.12%] [G loss: 0.069396, mse: 0.067804]
4759 [D loss: 0.392070, acc: 84.38%] [G loss: 0.063913, mse: 0.062090]
4760 [D loss: 0.357826, acc: 85.94%] [G loss: 0.068119, mse: 0.066471]
4761 [D loss: 0.426540, acc: 82.81%] [G loss: 0.068935, mse: 0.067051]
4762 [D loss: 0.446442, acc: 79.69%] [G loss: 0.067542, mse: 0.066124]
4763 [D loss: 0.431313, acc: 81.25%] [G loss: 0.079211, mse: 0.077929]
4764 [D loss: 0.461517, acc: 81.25%] [G loss: 0.067429, mse: 0.065906]
4765 [D loss: 0.407409, acc: 84.38%] [G loss: 0.074418, mse: 0.073014]
4766 [D loss: 0.437937, acc: 84.38%] [G loss: 0.064964, mse: 0.063389]
4767 [D loss: 0.375632, acc: 82.81%] [G loss: 0.065302, mse: 0.063414]
4768 [D loss: 0.404977, acc: 85.94%] [G loss: 0.070237, mse: 0.068802]
4769 [D loss: 0.419012, acc: 78.12%] [G loss: 0.066235, mse: 0.064500]
4770 [

4873 [D loss: 0.456977, acc: 71.88%] [G loss: 0.077174, mse: 0.075669]
4874 [D loss: 0.552194, acc: 73.44%] [G loss: 0.075495, mse: 0.074269]
4875 [D loss: 0.446513, acc: 76.56%] [G loss: 0.062698, mse: 0.060948]
4876 [D loss: 0.467661, acc: 78.12%] [G loss: 0.069550, mse: 0.068202]
4877 [D loss: 0.351436, acc: 84.38%] [G loss: 0.064236, mse: 0.062476]
4878 [D loss: 0.431828, acc: 81.25%] [G loss: 0.066453, mse: 0.064640]
4879 [D loss: 0.498124, acc: 76.56%] [G loss: 0.067113, mse: 0.065598]
4880 [D loss: 0.421435, acc: 78.12%] [G loss: 0.079562, mse: 0.078107]
4881 [D loss: 0.474767, acc: 76.56%] [G loss: 0.055989, mse: 0.054051]
4882 [D loss: 0.476284, acc: 79.69%] [G loss: 0.071699, mse: 0.070194]
4883 [D loss: 0.420337, acc: 78.12%] [G loss: 0.067628, mse: 0.066199]
4884 [D loss: 0.398761, acc: 79.69%] [G loss: 0.067205, mse: 0.065608]
4885 [D loss: 0.390659, acc: 82.81%] [G loss: 0.071770, mse: 0.069980]
4886 [D loss: 0.364264, acc: 84.38%] [G loss: 0.077096, mse: 0.075545]
4887 [

4990 [D loss: 0.405744, acc: 78.12%] [G loss: 0.064724, mse: 0.062834]
4991 [D loss: 0.451939, acc: 79.69%] [G loss: 0.073378, mse: 0.071761]
4992 [D loss: 0.472787, acc: 76.56%] [G loss: 0.067866, mse: 0.066117]
4993 [D loss: 0.408566, acc: 89.06%] [G loss: 0.070381, mse: 0.068800]
4994 [D loss: 0.447603, acc: 75.00%] [G loss: 0.070532, mse: 0.069078]
4995 [D loss: 0.400343, acc: 84.38%] [G loss: 0.063642, mse: 0.061996]
4996 [D loss: 0.506928, acc: 79.69%] [G loss: 0.074034, mse: 0.072653]
4997 [D loss: 0.386728, acc: 84.38%] [G loss: 0.074504, mse: 0.072769]
4998 [D loss: 0.432002, acc: 82.81%] [G loss: 0.061785, mse: 0.060272]
4999 [D loss: 0.469720, acc: 78.12%] [G loss: 0.078040, mse: 0.076671]
